In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

In [6]:
from ConuForecast.src.graph_utils import GraphInit, DBconnector
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

conn = DBconnector('172.18.0.1', 5555, 'base-ina', 'postgres', 'postgres')

ELAPSED_TIME = '2017-01-01 12:00:00'
MODEL = 'model_007'
EVENT = '007'
PRECIP = 'precipitation_007'

graph = GraphInit(elapsed_time=ELAPSED_TIME, model=MODEL, event=EVENT, precip=PRECIP, conn=conn)

In [11]:
df_plot_0= graph.timeseries('NODO1200', ['rainfall'])
df_plot_0 = df_plot_0.resample('1H').mean()
df_plot_0 = df_plot_0[df_plot_0.index  < pd.to_datetime('2019-10-13 01:00:00')]

df_plot_1 = graph.timeseries('NODO1200', ['depth_above_invert', 'rainfall'])
df_plot_1 = df_plot_1[df_plot_1.index  < pd.to_datetime('2019-10-13 01:00:00')]

df_plot_2 = graph.timeseries('NODO2792', ['depth_above_invert'])
df_plot_2 = df_plot_2[df_plot_2.index  < pd.to_datetime('2019-10-13 01:00:00')]



subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.bar(df_plot_0, y=['rainfall'])#, render_mode="webgl",)
fig2 = px.line(df_plot_1, y='depth_above_invert')
fig2.update_traces(line={'width':5, 'color':'#125AEF'})
fig3 = px.line(df_plot_2, y='depth_above_invert')
fig3.update_traces(line={'width':5, 'color':'#3C9249'})

fig2.update_traces(yaxis="y2")
fig3.update_traces(yaxis="y2")


subfig.add_traces(fig.data + fig2.data + fig3.data)
subfig['layout']['yaxis1'].update(title='Precipitation intensity (mm/h)',range=[0, 60], autorange='reversed')
subfig['layout']['yaxis2'].update(title='Water level (m)', range=[0, 1], autorange=False)
subfig.for_each_trace(lambda t: t.update(marker=dict(color=['black'])))
subfig['layout']['xaxis'].update(title='', tickformat='%d-%b %Hh')
subfig['layout'].update(plot_bgcolor='white', font={'size':25})#, template='plotly_white')
subfig.update_xaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
subfig.update_yaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
subfig.update_xaxes(ticks="inside", tickwidth=2, tickcolor='black', ticklen=10)
subfig.update_yaxes(ticks="inside", tickwidth=2, tickcolor='black', ticklen=10)

subfig['layout'].update(height=600, width=1200)
subfig.update_layout(showlegend=False)
subfig.update_layout(
    shapes=[{'type': 'line',
             'y0':0, 'y1': 1,
             'x0':graph.elapsed_time, 'x1':graph.elapsed_time,
             'xref':'x', 'yref':'paper',
             'line':{
                 'color':"Red",
                 'width':3,
                 'dash':"dash"
             }}],)